# 0. Configuring the project
Before getting started, we will install some external modules crucial to the execution of our project.
 

In [ ]:
%pip install python_speech_features                                                           
%pip install youtube_dl
%pip install sklearn

# 1. Imports
After installing the aforementioned packages, we're now ready to import them into our project file

In [2]:
from python_speech_features import mfcc                                           # Aids in extracting and operating on sound features 
import youtube_dl                                                                 # Youtube downloader to download songs from a youtube playlist
from sklearn.cluster import KMeans
import pprint
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
from collections import defaultdict
import os

from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

import sklearn
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

## Loading up the dataset

In [3]:
# directory = "/mnt/c/Users/nikhi/Desktop/final_project/genres"
# f= open("my1.dat" ,'wb')
# f2= open("my2.dat", 'wb')
# i=0
# for folder in os.listdir(directory):
#     i+=1
#     if i==11 :
#         break   
#     for file in os.listdir(directory+"/"+folder):  
#         (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
#         mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
#         covariance = np.cov(np.matrix.transpose(mfcc_feat))
#         mean_matrix = mfcc_feat.mean(0)
#         feature = (mean_matrix , covariance , i)
#         pickle.dump(feature , f)
#         feature1 = (mean_matrix, covariance)
#         pickle.dump(feature1, f2)
# f.close()

In [4]:
def loadDataset(filename, dataset):
    with open(filename , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break   

dataset1 = []      # Dataset with labels 
dataset2 = []      # Dataset without labels
loadDataset("my1.dat", dataset1)
loadDataset("my2.dat", dataset2) 

## Exploratory data analysis

In [8]:
dataset1_np = np.array(dataset1) 
print(dataset1_np.shape)

print(len(dataset2))

# Current shape of dataset2: 1000 elements each containing 2 elements: #1: 13 elements #2: 13 elements each containing 13 elements
print(len(dataset2[0][1][1]))

# Declare a new numpy array for dataset2
dataset2_np = np.empty((len(dataset2), 182))

# print(len(dataset2[5][0])) # Should be 13
# print(len(dataset2[5][1])) # Should be 13
# print(len(dataset2[5][1][1])) # should be 13

# Flattening the dataset
for i in range(len(dataset2)):
    # Declare a temp array that we will append to the main np array
    array_i = []
    for j in range(len(dataset2[i][0])):
        array_i.append(dataset2[i][0][j])
    for k in range(len(dataset2[i][1])):
        for k2 in range(len(dataset2[i][1][1])):
            array_i.append(dataset2[i][1][k][k2])
    array_np = np.array(array_i)
    np.append(dataset2_np, array_np)


print(dataset2_np.shape)

# Preparing Target Data
target = []
for i in range(0, 1000):
    target.append(dataset1_np[i][2])
del target[-1]
del target[-1]
target_np = np.array(target)

print(target_np.shape)
print(np.unique(target_np))

/home/nikhilbhave9/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(1000, 3)
998
13
(998, 182)
(998,)
[ 1  2  3  4  5  6  7  8  9 10]


There are 10 genres and each genre has a 100 songs each. Thus, there are a 1000 audio samples in total. 
Each song has 3 "features" associated with it associated with it. Each of these 3 features is represented using a tuple of varying sizes. 
- Feature #1: Mean_matrix -> A flat array of size 13
- Feature #2: Covariance  -> A 2-D matrix of size 13 x 13
- Feature #3: Label       -> An int representing the correct label for the audio sample 

However, we do not need the labels "feature" while fitting an SVM model to our data. We extract the labels from this dataset and add them to a "target" array. 

Now, we create another dataset, called dataset2, without labels as the third feature. In order to work with this second dataset and feed it to .fit method, we first need to flatten the data. Thus, we will now have, for each of the 1000 songs, one flat 1-D array of size 13 + (13 * 13) = 13 + 169 = 182. Thus, our final dataset2 shape should be (1000, 182)


### Cleaning the dataset 
Let's check for inconsistent values

In [9]:
dataset2_np.shape[0]
dataset2_np_copy = dataset2_np

In [10]:
for n in range(0, dataset2_np.shape[0]):
    for m in range(len(dataset2_np[n])):
        if np.isnan(dataset2_np[n][m]):
            print("NaN spotted")

No NaN spotted

## Test-Train

In [11]:
# Splice the dataset into X => 2 main features and Y => Correct labels
X_train, X_test, y_train, y_test = train_test_split(dataset2_np, target_np, test_size=0.2,random_state=109) # 80% training and 20% test

# Print shape of X_train 
print("Shape of X_train: ")
print(X_train.shape)

# Print shape of X_test 
print("Shape of X_test: ")
print(X_test.shape)

# Print shape of y_train 
print("Shape of y_train: ")
print(y_train.shape)

# Print shape of y_test 
print("Shape of y_test: ")
print(y_test.shape)

Shape of X_train: 
(798, 182)
Shape of X_test: 
(200, 182)
Shape of y_train: 
(798,)
Shape of y_test: 
(200,)


## Fitting our data to an SVM model

In [21]:
#Create SVM Classifier
classifier = svm.SVC(kernel='rbf')

#Train the model using the training sets
classifier.fit(X_train, y_train)

print(X_train)

#Predict the response for test dataset
y_pred = classifier.predict(X_test)

print(y_pred)

[[ 2.01600e+02 -5.40660e+02  1.75800e+01 ... -2.67370e+02  3.31330e+03
   1.53358e+03]
 [ 1.92860e+02  1.31506e+03  1.45380e+03 ...  2.08100e+02 -2.93290e+02
  -4.21980e+02]
 [-3.36060e+02 -7.50400e+01 -1.05860e+02 ... -1.20670e+02  1.15140e+02
   1.29090e+02]
 ...
 [ 2.71800e+02  2.85270e+02  2.03920e+02 ... -1.07000e+00 -1.42210e+02
  -1.28550e+02]
 [-5.52280e+02 -2.62270e+03 -1.44277e+03 ... -1.59469e+03  1.05503e+03
   3.53966e+03]
 [ 1.28950e+02 -1.34187e+03 -6.82910e+02 ... -2.20302e+03  5.45740e+02
   2.92195e+03]]
[ 8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8 10  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8 10  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8 

/home/nikhilbhave9/.local/lib/python3.6/site-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


## Model Accuracy

In [22]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", metrics.precision_score(y_test, y_pred, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", metrics.recall_score(y_test, y_pred,average="micro"))

Accuracy: 0.08
Precision: 0.08
Recall: 0.08


## KMC

In [23]:
#Create SVM Classifier
classifier1 = KMeans(n_clusters=10)

print(X_train)

#Train the model using the training sets
classifier1.fit(X_train)

#Predict the response for test dataset
y_pred = classifier1.predict(X_test)

print(y_pred)

[[ 2.01600e+02 -5.40660e+02  1.75800e+01 ... -2.67370e+02  3.31330e+03
   1.53358e+03]
 [ 1.92860e+02  1.31506e+03  1.45380e+03 ...  2.08100e+02 -2.93290e+02
  -4.21980e+02]
 [-3.36060e+02 -7.50400e+01 -1.05860e+02 ... -1.20670e+02  1.15140e+02
   1.29090e+02]
 ...
 [ 2.71800e+02  2.85270e+02  2.03920e+02 ... -1.07000e+00 -1.42210e+02
  -1.28550e+02]
 [-5.52280e+02 -2.62270e+03 -1.44277e+03 ... -1.59469e+03  1.05503e+03
   3.53966e+03]
 [ 1.28950e+02 -1.34187e+03 -6.82910e+02 ... -2.20302e+03  5.45740e+02
   2.92195e+03]]


/home/nikhilbhave9/.local/lib/python3.6/site-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:314: RuntimeWarning: invalid value encountered in add
  distances += XX
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:314: RuntimeWarning: invalid value encountered in add
  distances += XX
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:314: RuntimeW

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/cluster/_kmeans.py:388: RuntimeWarning: overflow encountered in square
  labels, center_shift, n_threads)
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/nikhilbhave9/.local/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:314: RuntimeWarning: invalid value encountered in add
  distances += XX
/home/nikhilbhave9/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  import sys


In [24]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", metrics.precision_score(y_test, y_pred, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", metrics.recall_score(y_test, y_pred,average="micro"))

Accuracy: 0.0
Precision: 0.0
Recall: 0.0
